**Objective: Clean the data**

Souce of the data https://www.kaggle.com/competitions/conversion-rate-prediction/data

In [6]:
import pandas as pd
from deep_translator import GoogleTranslator
import time
from googletrans import Translator, constants
from pprint import pprint

In [8]:
translator = Translator()
def translate(phrase):
    return translator.translate(phrase,src='es',dest='en').text

In [9]:
#Read the data into DataFrames
train = pd.read_parquet('src/data/train')
test = pd.read_parquet('src/data/test')

In [16]:
#merge the two datasets(we do not need them seperated)
df = pd.concat([train,test])

rename_mapper = {
    'boosted':'boosted',
    'full_name':'taxonomy_name',
    'free_shipping':'free_shipping',
    'conversion':'conversion',
    'is_pdp':'is_pdp',
    'logistic_type':'logistic_type',
    'original_price':'original_price',
    'platform':'platform',
    'price':'price',
    'print_position':'page_location',
    'print_server_timestamp': 'display_date',
    'title':'title'
    }

cols = list(rename_mapper.keys())

In [17]:
df = df[cols].rename(rename_mapper)
df.head(5)

,boosted,full_name,free_shipping,conversion,is_pdp,logistic_type,original_price,platform,price,print_position,print_server_timestamp,title
0,False,Celulares y Teléfonos -> Celulares y Smartphones,True,False,False,cross_docking,29999,/web/desktop,20999,1717,2020-03-06T00:19:30.735-0400,Motorola One Action 128 Gb Denim Blue 4 Gb Ram
1,False,Deportes y Fitness -> Zapatillas,True,False,False,drop_off,6700,/mobile/android,3999,311,2020-04-01T21:20:11.738-0400,Zapatillas Hovr Sonic 2 Under Armour
2,False,"Ropa y Accesorios -> Pantalones, Jeans y Joggings",False,False,False,fulfillment,1490,/mobile/android,1341,1686,2020-04-08T18:38:48.360-0400,Pantalon Mujer Vestir Elastizado Tiro Alto Tal...
3,False,Juegos y Juguetes -> Vehículos de Juguete -> V...,True,False,False,fulfillment,3498,/web/mobile,3007,12,2020-04-25T22:01:19.829-0400,Trencity Pista N°3 - Kit Turbo Elevación + Pu...
4,False,Computación -> Componentes de PC -> Memorias RAM,False,False,False,cross_docking,1248,/mobile/android,998,511,2020-03-10T13:20:56.633-0400,Memoria Ram Ddr2 2gb 800mhz Samsung - Intel Y Amd


In [18]:
df['device_type'] = df['platform'].apply(lambda x: x.split('/')[2])
df['platform'] = df['platform'].apply(lambda x: x.split('/')[1])
df['taxonomy_level'] = df['full_name'].apply(lambda x: len(x.split('->'))+1)

In [19]:
df['taxonomy_level']= df['full_name'].str.count("->") + 1
df['taxonomy_level1'] = df['full_name'].str.split('->').str[0]

In [23]:
tax_levels = df['taxonomy_level1'].unique()
mapper = {}
for i in tax_levels.tolist():
    mapper[i] = translate(i)

In [25]:
df['taxonomy_level1'] = df['taxonomy_level1'].apply(lambda x: mapper[x])

In [26]:
df['conversion'].fillna(0, inplace = True)
df['is_pdp'].fillna(0, inplace = True)
df['free_shipping'].fillna(0, inplace = True)

In [27]:
true_false_cols = ['free_shipping','conversion','is_pdp']
for col in true_false_cols:
    df[col] = df[col].apply(lambda x: int(x))

In [28]:
df.to_parquet('src/data/data')

In [29]:
df

,boosted,full_name,free_shipping,conversion,is_pdp,logistic_type,original_price,platform,price,print_position,print_server_timestamp,title,device_type,taxonomy_level,taxonomy_level1
0,False,Celulares y Teléfonos -> Celulares y Smartphones,1,0,0,cross_docking,29999,web,20999,1717,2020-03-06T00:19:30.735-0400,Motorola One Action 128 Gb Denim Blue 4 Gb Ram,desktop,2,Cell phones and Telephones
1,False,Deportes y Fitness -> Zapatillas,1,0,0,drop_off,6700,mobile,3999,311,2020-04-01T21:20:11.738-0400,Zapatillas Hovr Sonic 2 Under Armour,android,2,Sports and Fitness
2,False,"Ropa y Accesorios -> Pantalones, Jeans y Joggings",0,0,0,fulfillment,1490,mobile,1341,1686,2020-04-08T18:38:48.360-0400,Pantalon Mujer Vestir Elastizado Tiro Alto Tal...,android,2,Clothes and accessories
3,False,Juegos y Juguetes -> Vehículos de Juguete -> V...,1,0,0,fulfillment,3498,web,3007,12,2020-04-25T22:01:19.829-0400,Trencity Pista N°3 - Kit Turbo Elevación + Pu...,mobile,4,Games and toys
4,False,Computación -> Componentes de PC -> Memorias RAM,0,0,0,cross_docking,1248,mobile,998,511,2020-03-10T13:20:56.633-0400,Memoria Ram Ddr2 2gb 800mhz Samsung - Intel Y Amd,android,3,computing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19206,False,"Hogar, Muebles y Jardín -> Bazar y Cocina -> V...",0,0,0,fulfillment,1550,mobile,1395,2665,2020-04-12T14:35:44.784-0400,Frasco Dispenser Vidrio C/relieve C/soporte C/...,android,5,"Home, Furniture and Garden"
19207,False,Belleza y Cuidado Personal -> Tratamientos de ...,0,0,0,cross_docking,635,web,577,25,2020-04-03T17:32:53.035-0400,Electro Estimulador Abdominal Reductor Fitness...,mobile,5,Beauty and Personal Care
19208,False,"Electrónica, Audio y Video -> Audio -> Auricul...",0,0,0,fulfillment,990,mobile,869,87,2020-04-01T22:51:08.898-0400,Auricular Bluetooth Vincha Micro Sd Radio Fm I...,android,3,"Electronics, Audio and Video"
19209,False,Juegos y Juguetes -> Muñecos y Muñecas -> Muñe...,0,0,0,drop_off,1719,mobile,1203,3751,2020-04-08T11:32:27.563-0400,Transformers Generations Dreadwind (1587),android,3,Games and toys
